In [ ]:
%%bash
MINICONDA_INSTALLER_SCRIPT=Miniconda3-4.5.4-Linux-x86_64.sh
MINICONDA_PREFIX=/usr/local
wget https://repo.continuum.io/miniconda/$MINICONDA_INSTALLER_SCRIPT
chmod +x $MINICONDA_INSTALLER_SCRIPT
./$MINICONDA_INSTALLER_SCRIPT -b -f -p $MINICONDA_PREFIX

In [ ]:
!which conda # 返回/usr/local/bin/conda
!conda --version # 返回4.5.4
!which python # 依旧是/usr/local/bin/python
!python --version # 返回 Python 3.6.5 :: Anaconda, Inc.

In [ ]:
!conda install --channel defaults conda python=3.6 --yes
!conda update --channel defaults --all --yes
!conda --version # now returns conda 4.10.3
!python --version # now returns Python 3.6.13 :: Anaconda, Inc.

In [ ]:
from pprint import pprint
import sys
_ = (sys.path
        .append("/usr/local/lib/python3.6/site-packages"))
pprint(sys.path)

In [ ]:
!conda create --name bcdm -y python=3.6

In [48]:
!source activate bcdm

In [57]:
!conda env list

# conda environments:
#
base                  *  /usr/local
bcdm                     /usr/local/envs/bcdm



In [ ]:
# this installs the right pip and dependencies for the fresh python
!conda install -y ipython pip
!pip install ninja yacs cython matplotlib tqdm opencv-python imageio mmcv

# follow PyTorch installation in https://pytorch.org/get-started/locally/
# we give the instructions for CUDA 9.2
!conda install pytorch==1.2.0 torchvision==0.4.0 cudatoolkit=9.2 -c pytorch

In [ ]:
!conda install -h

In [ ]:
!conda install -n base ipykernel  #直接指定环境安装ipykernel

In [ ]:
!unzip /content/BCDM-1.0.zip

In [ ]:
!pip install kaggle

In [ ]:
import json
token = {"username":"shaojiezhang04","key":"af91bfc4c77fc89fdfdcfc59526d3c1f"}
with open('/content/kaggle.json', 'w') as file:
  json.dump(token, file)
!mkdir -p ~/.kaggle
!cp /content/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle config set -n path -v /content

In [ ]:
!kaggle datasets download -d lliu374/gta5-part1

In [ ]:
!kaggle datasets download -d lqdisme/cityscapes

In [ ]:
!unzip /content/datasets/lliu374/gta5-part1/gta5-part1.zip -d /content/BCDM-1.0/segmentation/datasets/gta5

In [ ]:
!unzip /content/datasets/lqdisme/cityscapes/cityscapes.zip -d /content/BCDM-1.0/segmentation/datasets/cityscapes

In [ ]:
!ln -s /content/BCDM-1.0/segmentation/datasets/gta5
!ln -s /content/BCDM-1.0/segmentation/datasets/cityscapes
%cd /content/BCDM-1.0/segmentation
!pwd
!python /content/BCDM-1.0/segmentation/datasets/generate_gta5_label_info.py -d datasets/gta5 -o datasets/gta5/

In [ ]:
!ls -ln

In [ ]:
import torch
 
print(torch.__version__)
print(torch.cuda.is_available())

In [ ]:
%cd /content/BCDM-1.0/segmentation
!python test_bcdm.py -cfg configs/deeplabv2_r101_bcdm.yaml resume /content/g2c_r101_model_iter016000.pth

In [5]:
%cd /content/BCDM-1.0/segmentation

/content/BCDM-1.0/segmentation


In [ ]:
!python -m torch.distributed.launch --nproc_per_node=1 /content/BCDM-1.0/segmentation/train_bcdm.py -cfg /content/BCDM-1.0/segmentation/configs/deeplabv2_r101_bcdm.yaml OUTPUT_DIR /data1/TL/DA-Seg/FADA/results/bcdm END 0

2021-12-12 07:39:51,381 FADA INFO: Using 1 GPUs
2021-12-12 07:39:51,381 FADA INFO: Namespace(config_file='/content/BCDM-1.0/segmentation/configs/deeplabv2_r101_bcdm.yaml', distributed=False, local_rank=0, opts=['OUTPUT_DIR', '/data1/TL/DA-Seg/FADA/results/bcdm', 'END', '0'], skip_test=False)
2021-12-12 07:39:51,381 FADA INFO: Loaded configuration file /content/BCDM-1.0/segmentation/configs/deeplabv2_r101_bcdm.yaml
2021-12-12 07:39:51,381 FADA INFO: 
MODEL:
  NAME: "deeplab_resnet101"
  WEIGHTS: "https://download.pytorch.org/models/resnet101-5d3b4d8f.pth"
  FREEZE_BN: True
DATASETS:
  SOURCE_TRAIN: "gta5_train"
  TARGET_TRAIN: "cityscapes_train"
  TEST: "cityscapes_val"
INPUT:
  BRIGHTNESS: 0.5
  CONTRAST: 0.5
  SATURATION: 0.5
  HUE: 0.2
#   PIXEL_MEAN: [103.939, 116.779, 123.68]
#   PIXEL_STD: [1., 1., 1.]
#   TO_BGR255: True
SOLVER:
  BASE_LR: 2.5e-4
  BASE_LR_D: 0.0001
  MAX_ITER: 62500
  STOP_ITER: 40000
  CHECKPOINT_PERIOD: 2000
  BATCH_SIZE: 2
  NUM_K: 2
  CDD_LOSS: 0.01
  ENT_LO

In [12]:
!rm -rf /content/datasets/lqdisme